In [1]:
import tweepy
import pandas as pd
import preprocessor as p
from watson_developer_cloud import PersonalityInsightsV3
from langdetect import detect_langs
from langid.langid import LanguageIdentifier, model
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import Row
import time
#import pixiedust
from pyspark import SparkContext
import itertools
from pyspark.sql.functions import *

In [2]:
#pixiedust.installPackage("https://github.com/sairam95/jar_file1/blob/master/spark-datasource-rest_2.11-2.1.0-SNAPSHOT.jar?raw=true")
#%pixiedustLog -l debug
#pixiedust.installPackage("https://spark.bluemix.net/tenant/data/libs/spark-datasource-rest_2.11-2.1.0-SNAPSHOT.jar")
#pixiedust.installPackage("https://github.com/sairam95/jar_file1/blob/master/spark-datasource-rest_2.11-2.1.0-SNAPSHOT.jar?raw=True")

#pixiedust.printAllPackages()

### Reading data from the html table

In [3]:
actors = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/celebrities/actor/page-1-2/")[0]
sports_star = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/celebrities/sport-star/page-1-2/")[0]
writer = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/celebrities/writer/page-1-2/")[0]
politicians = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/society/politics/page-1-2/")[0]

#### Renaming the columns in above data frames

In [4]:
data = [actors, sports_star, writer, politicians]
for item in data:
    item.columns = ['rank', 'celebrity', 'followings', 'followers']

Merging all the categories of celebrities to one

In [5]:
df = pd.concat(data)
celebrities = df.celebrity.dropna(axis=0, how='all')

In [6]:
celebrities.head()

0    1  Amitabh Bachchan (@SrBachchan)
1          2  Shah Rukh Khan (@iamsrk)
2    3  Salman Khan (@BeingSalmanKhan)
3       4  Akshay Kumar (@akshaykumar)
4          5  Aamir Khan (@aamir_khan)
Name: celebrity, dtype: object

#### cleaning the df columns to obtain Twitter user names

In [7]:
celeb_list  = celebrities.str.extract('@(\S+)\)', expand = True)[0].tolist()

In [8]:
consumer_key = '2QTDFjrpIy71bwCuJXwgC85oD'
consumer_secret = '2sctC17IBpkigXU7TbzQnygSK3Uydit9yw4Pn0Xjhc5N8Jaftd'
access_token = '2891438354-ZrPa7xndtycr4DoNiG2JkdQXZ52WrZEIiIRyG3X'
access_secret = '92HfNTEzY9HNrbrHPcvYQPibFtA1BCtzRwPSiVTUETkhv'

### outputs only the Tweets of a specifc user

- following function uses LanguageIdentifier package for extracting the relevant tweets that have probability of being written in english greater than 0.9 
- Language Detection using langid package gives better accuracy in detecting English language but compile time is high. 


In [234]:
def detect_langid_relevant_tweets(screen_name):
    #Twitter only allows access to a users most recent tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    relevant_tweet = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200, lang='en', include_rts=False, exclude_replies = True)
    content_type = 'text/plain'
    content_language = None
    accept ='application/json'
    accept_language = None
    raw_scores = False
    consumption_preferences = False
    csv_headers = False
    
    #save most recent tweets
    
    alltweets.extend(new_tweets)
    #save the id of the oldest   less one
    oldest = alltweets[-1].id - 1
    
    

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest, lang='en', include_rts=False, exclude_replies = True)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
    
    for tweet in alltweets:
        # Data Cleaning 2> Extracting alphabetic and space text from tweets 
        twt = "".join(x for x in p.clean(tweet.text) if x.isalpha() or x.isspace())
        identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
        langProb = identifier.classify(twt)
        if langProb[0] == 'en' and float(langProb[1]) > 0.9:
            relevant_tweet.append(twt)
    return relevant_tweet

### Extracting relevant tweets Using Lang detect package

- Langdetect package has less compile time but accuracy of identifying English language is lesser.

In [9]:
def detect_langs_relevant_tweets(screen_name):
    #Twitter only allows access to a users most recent tweets with this method

    #authorize twitter, initialize tweepy
    start_time = time.time()
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    relevant_tweet = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200, lang='en', include_rts=False, exclude_replies = True)
    
    #save most recent tweets
    
    alltweets.extend(new_tweets)
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest, lang='en', include_rts=False, exclude_replies = True)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
    
    for tweet in alltweets:
        # Data Cleaning 2> Extracting alphabetic and space text from tweets 
        twt = "".join(x for x in p.clean(tweet.text) if x.isalpha() or x.isspace())
        twt = twt.strip()
        if (len(twt) > 0):
            if (str(detect_langs(twt)[0])[:2] =='en') & (float(str(detect_langs(twt)[0])[3:]) > 0.9):
                relevant_tweet.append(twt)
    
    return relevant_tweet

In [122]:
exec_time =0
for celeb in celeb_list[0:4]:
    start_time = time.time()
    c =detect_langs_relevant_tweets(celeb)
    exec_time = exec_time + (time.time()-start_time)
    print(exec_time)
    

23.29035449028015
89.24196600914001
110.99212670326233
113.45561408996582


In [605]:
## Get PI object using PI credentials - Aradhna
personality_insights = PersonalityInsightsV3(
version='2017-10-13',
username='04847f5d-afb4-4b08-8ed1-ae8ac7a56ac9',
password='8jniPKnFjDYO')

In [10]:
# Ram PI Credentials 
personality_insights = PersonalityInsightsV3(
version='2017-10-13',
username='f2264642-7379-488b-9154-e1373384cbc0',
password='rUBImsJrWStQ')

In [14]:
## Call Personality Insights on the tweets for the user
def getPersonality(tweets):
    # get tweets by user
    start_time = time.time()
    if tweets == None:
        profile = None
    else:
        tweets_content = ' '.join(tweets)
        # UTF-8 encoding
        twt = tweets_content.encode('utf-8')
        
        # call PI to get personality profile
        try:
            profile = personality_insights.profile(twt, content_type = 'text/plain', content_language = None,
                                           accept ='application/json', accept_language = None, raw_scores = False,
                                           consumption_preferences = False, csv_headers = False)
            
        except Exception:
            profile = None
    print('Execution time for getPersonality function: ', time.time()-start_time)
    return profile

## Extract OCEAN percentiles from PI data
def extractOCEANtraits(profile):
    start_time = time.time()
    if profile == None:
        openness = None
        conscientiousness = None
        extraversion = None
        agreeableness = None
        neuroticism = None
    else:
        personality = profile['personality']
        openness = personality[0]['percentile']
        conscientiousness = personality[1]['percentile']
        extraversion = personality[2]['percentile']
        agreeableness = personality[3]['percentile']
        neuroticism = personality[4]['percentile']
        
    #print('Execution time for extractOCEANtraits function: ', time.time()-start_time)
    return openness, conscientiousness, extraversion, agreeableness, neuroticism

## Combine function calls for a user
def getPItraits(user):
    
    pitraits = {}
    try:
        ### Uncomment the line below to use FASTER language detection.
        
        #tweets = LanguageIdentifier_relevant_tweets(user)
        
        ### The line below gives a more ACCURATE language detection.
        tweets = detect_langs_relevant_tweets(user)
        # run PI profile on extracted tweets
        profile = getPersonality(tweets)
        # extract OCEAN traits
        traits = extractOCEANtraits(profile)
        if traits:
            # writing the personality profile of a user into dictionary
            OCEANTraits = dict(zip(['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism'],traits))
            return dict([(user,OCEANTraits)])
        else:
            return None
    except Exception:
        return None

#### looping around the celebrities list and converting the Personality traits result dictionary into a data frame


In [15]:
pitraits_dict = {}
### Restricted the celeb list to 4 to display final output. 
start = time.time()
for celeb in celeb_list[0:5]:
    PItraits = getPItraits(celeb)
    if PItraits is not None:
        #print(celeb)
        pitraits_dict.update(getPItraits(celeb))
    else:
        pass
traits_df = pd.DataFrame(pitraits_dict)
traits_df = traits_df.transpose()
traits_df
print('Time without using Parallel processing: ', (time.time() - start))

Execution time for getPersonality function:  0.5358307361602783
Execution time for getPersonality function:  0.5896203517913818
Execution time for getPersonality function:  1.3133835792541504
Execution time for getPersonality function:  1.3330190181732178
Execution time for getPersonality function:  0.4638671875
Execution time for getPersonality function:  0.47318029403686523
Execution time for getPersonality function:  0.13445472717285156
Execution time for getPersonality function:  0.13078761100769043
Execution time for getPersonality function:  0.3430323600769043
Execution time for getPersonality function:  0.5589053630828857
Time without using Parallel processing:  261.263188123703


## Modified code for accessing Rest API service using pyspark

In [608]:
def tweet_cleaning(y):
        # Data Cleaning 2> Extracting alphabetic and space text from tweets 
    twt = "".join(x for x in p.clean(y.text) if x.isalpha() or x.isspace())
    twt = twt.strip()
    if (len(twt) > 0):
        if (str(detect_langs(twt)[0])[:2] =='en') & (float(str(detect_langs(twt)[0])[3:]) > 0.9):
            return twt
        else:
            return ""
    else:
        return ""

In [609]:
def detect_langs_relevant_tweets(screen_name):
    #Twitter only allows access to a users most recent tweets with this method

    #authorize twitter, initialize tweepy
    start_time = time.time()
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)

    alltweets = [] #initialize a list to hold all the tweepy Tweets

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200, lang='en', include_rts=False, exclude_replies = True)  
    alltweets.extend(new_tweets)  #save most recent tweets
    oldest = alltweets[-1].id - 1 #save the id of the oldest tweet less one
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest, lang='en', include_rts=False, exclude_replies = True)

        alltweets.extend(new_tweets)  #save most recent tweets
    
        oldest = alltweets[-1].id - 1  #update the id of the oldest tweet less one
    
    #print(alltweets)
    alltweets_rdd = sc.parallelize(alltweets)
    relevant_tweet_rdd = alltweets_rdd.map(tweet_cleaning)
        
    #print('Execution time for relevant_tweets function: ', time.time()-start_time)
    return relevant_tweet_rdd.collect()

In [610]:
celeb_tweets_dict = {}
pitraits = {}

In [611]:
### Restricted the celeb list to 4 to display final output.
count, exec_time = 0, 0
start = time.time()
for celeb in celeb_list[0:5]:
    count = count +1
    start_time = time.time()
    tweets= detect_langs_relevant_tweets(celeb)
    #print('Count of celebrities whose tweets are extracted: ', count)
    # get tweets by user
    tweets_content = ' '.join(tweets)
    exec_time = exec_time + (time.time()-start_time)
    print(exec_time)
    # UTF-8 encoding
    #twt = tweets_content.encode('utf-8')    
            # call PI to get personality profile
    #print(twt)
    
    celeb_tweets_dict.update({celeb:tweets_content})

19.573287963867188
68.83662867546082
82.5223159790039
84.33840417861938
89.21816682815552


In [612]:
lst = celeb_tweets_dict.keys()
celeb_dict = sc.parallelize(lst) #Creates an RDD of list of celebrites


    #Key value pair of celebrity name and his Watson Insights

    ##Code to the rest APi service
pi_time = time.time()
celeb_pi = celeb_dict.map(lambda celeb:(celeb, personality_insights.profile(celeb_tweets_dict[celeb], content_type = 'text/plain', content_language = None,
                                           accept ='application/json', accept_language = None, raw_scores = False,
                                           consumption_preferences = False, csv_headers = False)))


    ##code to extract the necessary part from the json format
    #converting the pi insights into a dataframe
OCEAN_Traits = celeb_pi.map(lambda x: Row(Celebrity = x[0], openness = x[1]['personality'][0]['percentile'], conscientiousness = x[1]['personality'][1]['percentile'], 
                                   extraversion = x[1]['personality'][2]['percentile'], agreeableness = x[1]['personality'][3]['percentile'],
                                       neuroticism = x[1]['personality'][4]['percentile']))

    ##Merging the data into a dataframe
type(OCEAN_Traits)
OCEAN_Traits = sqlContext.createDataFrame(OCEAN_Traits)


print('Twitter tweets extraction time for all celebrities: ', exec_time)    
OCEAN_Traits.show()
print('Total time for PI analysis: ', time.time()-pi_time)


Twitter tweets extraction time for all celebrities:  89.21816682815552
+---------------+------------------+------------------+-------------------+-------------------+-------------------+
|      Celebrity|     agreeableness| conscientiousness|       extraversion|        neuroticism|           openness|
+---------------+------------------+------------------+-------------------+-------------------+-------------------+
|BeingSalmanKhan|0.6706261168601615|0.6809483726035818|0.15524348233633606| 0.5242585315880768| 0.3137146300929228|
|    akshaykumar|0.9999967737036528|0.9990511244875682| 0.9934734571307775|  0.978818820923842| 0.8728125334468189|
|         iamsrk|0.9843635833124369|0.7309454974693278| 0.5101603320672007| 0.5534924671151356|0.32683418096191347|
|     SrBachchan|0.7568636452306099|0.9151003533179586|0.31380546597503944| 0.7574107558227448| 0.8114497631929141|
|     aamir_khan|0.9553798951285026|0.8194044162955643| 0.5816580837189946|0.44617936260251917| 0.5373749483189343|
+

In [613]:
tweets = celeb_tweets_dict.values()

In [669]:
#updating actors list
actors = celeb_tweets_dict.keys()
actors = list(itertools.chain.from_iterable(itertools.repeat(x, 5) for x in actors))
actors_df = spark.createDataFrame(actors, schema=StringType())
actors_df = actors_df.selectExpr("value as actor")


In [670]:
actors_df.show()

+---------------+
|          actor|
+---------------+
|BeingSalmanKhan|
|BeingSalmanKhan|
|BeingSalmanKhan|
|BeingSalmanKhan|
|BeingSalmanKhan|
|    akshaykumar|
|    akshaykumar|
|    akshaykumar|
|    akshaykumar|
|    akshaykumar|
|         iamsrk|
|         iamsrk|
|         iamsrk|
|         iamsrk|
|         iamsrk|
|     SrBachchan|
|     SrBachchan|
|     SrBachchan|
|     SrBachchan|
|     SrBachchan|
+---------------+
only showing top 20 rows



In [388]:
#newDF = pd.DataFrame.from_dict(celeb_tweets_dict, orient='index')

In [393]:
#newDF.column = ['row']

In [394]:
#newDF

,0
row,
BeingSalmanKhan,A special thank you to and for the endless su...
akshaykumar,Hi guys extremely sorry about the delay will b...
iamsrk,My friend A man A parent This Sunday at pm on...
SrBachchan,T At the venue rehearsing for tomorrow event ...
aamir_khan,Hey guys watch the video and do the best you ...


In [396]:
#newDF = pd.DataFrame(newDF.row.str.split(' ',1).tolist(),columns = ['celebrity','value'])

In [671]:
tweet_rdd = sc.parallelize(tweets)

In [672]:
tweet_rdd.count()

5

In [673]:
from pyspark.sql.types import StringType
watsoninputDf = tweet_rdd.toDF(StringType())

In [674]:
watsoninputDf.show()

+--------------------+
|               value|
+--------------------+
| A special thank ...|
|Hi guys extremely...|
|My friend A man A...|
|T  At the venue r...|
| Hey guys watch t...|
+--------------------+



In [675]:
watsoninputDf.createOrReplaceTempView("watsoninputt")

In [676]:
watsonuri = 'https://gateway.watsonplatform.net/personality-insights/api/v2/profile'

In [677]:
# Now we create the parameter map to pass to the REST Data Source.
prmsWatson = { 'url' : watsonuri, 'input' : 'watsoninputt', 'inputContentType' :'text/plain', 'inputFormat' : 'text', 'userId' : 'f2264642-7379-488b-9154-e1373384cbc0','userPassword' : 'rUBImsJrWStQ', 'readTimeout' : '10000', 'connectionTimeout' : '2000', 'partitions' : '2'}

In [678]:
watsonDf = spark.read.format("org.apache.dsext.spark.datasource.rest.RestDataSource").options(**prmsWatson).load()

In [ ]:
watsonDf.cache()

In [ ]:
watsonDf.count()

In [679]:
watsonDf.printSchema()

root
 |-- output: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- processed_lang: string (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- tree: struct (nullable = true)
 |    |    |-- children: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- children: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- category: string (nullable = true)
 |    |    |    |    |    |    |-- children: array (nullable = true)
 |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |-- category: string (nullable = true)
 |    |    |    |    |    |    |    |    |-- children: array (nullable = true)
 |    |    |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |    |    |-- category: string (nullable = t

In [680]:
watsonDf.createOrReplaceTempView("watsonresulttbl")

In [681]:
watsonPersonalityDf.createOrReplaceTempView("watsonPersonalitytbl")

In [682]:
watsonPersonality2Df = spark.sql("select value, inline(children) from watsonPersonalitytbl")

In [683]:
watsonPersonality2Df.createOrReplaceTempView("watsonPersonality2tbl")

In [684]:
watsonPersonality3Df = spark.sql("select value, inline(children) from watsonPersonality2tbl where category='personality' ")

In [685]:
watsonPersonality3Df.createOrReplaceTempView("watsonPersonality3tbl")

In [686]:
spark.sql("select value,name,percentage from watsonPersonality3tbl").show()

+--------------------+-----------------+-------------------+
|               value|             name|         percentage|
+--------------------+-----------------+-------------------+
| A special thank ...|         Openness|0.31232384272828007|
| A special thank ...|Conscientiousness| 0.6870956327675607|
| A special thank ...|     Extraversion| 0.1462037421505351|
| A special thank ...|    Agreeableness| 0.6768484959942631|
| A special thank ...|  Emotional range| 0.5252973624289085|
|My friend A man A...|         Openness|0.33259758545955875|
|My friend A man A...|Conscientiousness| 0.7193596833200233|
|My friend A man A...|     Extraversion| 0.5050132104141529|
|My friend A man A...|    Agreeableness| 0.9819873802730845|
|My friend A man A...|  Emotional range| 0.5408504292541368|
| Hey guys watch t...|         Openness| 0.5328151749030463|
| Hey guys watch t...|Conscientiousness| 0.8206585028356688|
| Hey guys watch t...|     Extraversion| 0.5741938561631901|
| Hey guys watch t...|  

In [687]:
watsonPersonality3Df.show()

+--------------------+-----------+--------------------+-----------------+-----------------+-------------------+--------------------+
|               value|   category|            children|               id|             name|         percentage|      sampling_error|
+--------------------+-----------+--------------------+-----------------+-----------------+-------------------+--------------------+
| A special thank ...|personality|[[personality,Adv...|         Openness|         Openness|0.31232384272828007|        0.0508007798|
| A special thank ...|personality|[[personality,Ach...|Conscientiousness|Conscientiousness| 0.6870956327675607|       0.06470402876|
| A special thank ...|personality|[[personality,Act...|     Extraversion|     Extraversion| 0.1462037421505351|       0.04871731644|
| A special thank ...|personality|[[personality,Alt...|    Agreeableness|    Agreeableness| 0.6768484959942631|        0.0859550166|
| A special thank ...|personality|[[personality,Ang...|      Neurotic

In [688]:
actors_df.createOrReplaceTempView("actorstbl")

In [689]:
df = spark.sql("WITH CTE1 AS \
( \
    SELECT ROW_NUMBER() OVER(ORDER BY value) AS ROWNUM, * FROM watsonPersonality3tbl \
), \
CTE2 AS \
( \
    SELECT ROW_NUMBER() OVER (ORDER BY actor) AS ROWNUM, * FROM actorstbl \
) \
SELECT * \
FROM CTE1 FULL JOIN CTE2 ON CTE1.ROWNUM = CTE2.ROWNUM")

In [690]:
df.show()

+------+--------------------+-----------+--------------------+-----------------+-----------------+-------------------+--------------------+------+---------------+
|ROWNUM|               value|   category|            children|               id|             name|         percentage|      sampling_error|ROWNUM|          actor|
+------+--------------------+-----------+--------------------+-----------------+-----------------+-------------------+--------------------+------+---------------+
|     1| A special thank ...|personality|[[personality,Adv...|         Openness|         Openness|0.31232384272828007|        0.0508007798|     1|BeingSalmanKhan|
|     2| A special thank ...|personality|[[personality,Ach...|Conscientiousness|Conscientiousness| 0.6870956327675607|       0.06470402876|     2|BeingSalmanKhan|
|     3| A special thank ...|personality|[[personality,Act...|     Extraversion|     Extraversion| 0.1462037421505351|       0.04871731644|     3|BeingSalmanKhan|
|     4| A special tha

In [692]:
reshaped_df = df.groupby('actor').pivot('id').max('percentage')

In [693]:
reshaped_df.show()

+---------------+------------------+------------------+------------------+------------------+-------------------+
|          actor|     Agreeableness| Conscientiousness|      Extraversion|       Neuroticism|           Openness|
+---------------+------------------+------------------+------------------+------------------+-------------------+
|    akshaykumar|0.9819873802730845|0.7193596833200233|0.5050132104141529|0.5408504292541368|0.33259758545955875|
|         iamsrk|0.7644182943461479|0.9160682009778915| 0.316679897539034|0.7541127627403503| 0.8150558232245735|
|     SrBachchan|0.9563494138573354|0.8206585028356688|0.5741938561631901|0.4431156910544323| 0.5328151749030463|
|BeingSalmanKhan|0.6768484959942631|0.6870956327675607|0.1462037421505351|0.5252973624289085|0.31232384272828007|
|     aamir_khan|0.9999961838651272|0.9989964488083662|0.9925143353169668|0.9778249335629177| 0.8510684621344495|
+---------------+------------------+------------------+------------------+--------------